**Import needed packages/modules**

In [ ]:
# Cell 1
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
from scipy.integrate import solve_ivp

import pandas as pd

**Set simulation parameters and initial conditions**
1. The simulation will last **4** seconds
2. The rocket will start at height = 0 meters and launch from rest $v_0=0.0$
3. The mass of each stage of the rocket does not include the mass of its motor



In [ ]:
# Cell 2
tf = 4.0  # Simulation time (s)

G = 6.67430e-11  # Gravitational constant (m^3/kg/s^2)
M = 5.972e24  # Mass of the Earth (kg)
R = 6.371e6  # Radius of the Earth (m)

# For this model rocket
STAGE1_MASS = 0.0519  # kg
STAGE2_MASS = 0.1729  # kg

# For an F15 model rocket engine
ENGINE_MASS = 0.101  # kg
ENGINE_THRUST = 17.2  # Newtons
ENGINE_BURNOUT = 1.6  # seconds

**Define functions to return the instantaneous thurst and mass of the rocket**
1. The variable $t$ is the elapsed time (in seconds) since liftoff
2. The thrust of a motor is constant throughout its burn
3. The mass of each motor reduces linearly throughout its burn
4. At motor burnout, it will have zero remaining mass
5. There is no delay in the ignition of Stage 2 motor after booster ejection
6. Air friction is not considered in this model

In [ ]:
# Cell 3
def thrust_func(t):
    # returns thrust in Newtons
    if t < ENGINE_BURNOUT * 2:
        return ENGINE_THRUST
    return 0

def rocket_mass_func(t):
    # returns weight in kilograms
    if t <= ENGINE_BURNOUT:
        motor_mass = ENGINE_MASS * (ENGINE_BURNOUT - t) / ENGINE_BURNOUT
        return STAGE1_MASS + motor_mass + STAGE2_MASS + ENGINE_MASS
    if t <= ENGINE_BURNOUT * 2:
        motor_mass = ENGINE_MASS * (ENGINE_BURNOUT * 2 - t) / ENGINE_BURNOUT
        return STAGE2_MASS + motor_mass
    return STAGE2_MASS

print(f"Liftoff Thrust = {thrust_func(0):.2f} N")
print(f"Liftoff Mass = {rocket_mass_func(0):.2f} kg")

**Define the model function containing the differential equation:**\
$\large\frac{d^2s}{{dt}^2}=\frac{(F_{\text{thrust}}-F_g)}{m}$

$m$ (mass of rocket in kg) = depends on elapsed time

$F_{\text{thrust}}$ (in Newtons) = depends on elapsed time

$\large\ F_g=\frac{G\times M_E\times m}{(R_E+h)^2}$ where height h (in meters) depends on elapsed time

Using linked first order differential equations:

1. $\large\frac{dv}{dt}=\frac{(F_{\text{thrust}}-F_g)}{m}$

2. $\large\frac{ds}{dt}=v$

In [ ]:
# Cell 4
def model(t, state_vector):
    v, h = state_vector # h (height) = distance
    m = rocket_mass_func(t)
    F_thrust = thrust_func(t)
    F_gravity = G * M * m / (R + h) ** 2
    d_v = (F_thrust - F_gravity) / m
    d_h = v
    return d_v, d_h

**Use scipy's `solve_ivp()` to numerically estimate the ODE using the RKF45 Method**
1. We will limit the solver to a _maximum_ time step of $0.01$ second
2. The actual time values used will be returned by the solver
3. The solver will return the velocity and height each time value

In [ ]:
# Cell 5
sol = solve_ivp(model, (0, tf), [0, 0], max_step=0.01)
t = sol.t
v, h = sol.y

# Display the first 10 time and displacement values
pd.DataFrame({
    'Time (s)': t[:10],
    'Velocity (m/s)': v[:10],
    'Height (m)': h[:10],
})

**Plot the rocket velocity and altitude over time**\
Include the `mean velocity` and `maximum height` over <u>four</u> seconds of flight

In [ ]:
# Cell 6
plt.figure(figsize=(10, 4))
ax = plt.subplot(1, 2, 1)
ax.plot(t, v, color="blue", lw=2)
ax.set_title(f"Rocket Velocity - Mean: {np.mean(v):.2f} m/s")
ax.set_xlabel("Time (s)")
ax.set_ylabel("Velocity (m/s)")
ax.grid("on")
ax = plt.subplot(1, 2, 2)
ax.plot(t, h, color="orange", lw=2)
ax.set_title(f"Rocket Altitude - Max: {np.max(h):,.2f} m")
ax.set_xlabel("Time (s)")
ax.set_ylabel("Altitude (m)")
ax.grid("on")
plt.tight_layout()
plt.show()